# Задание.
* Подсчитать фичи для модели, используя только Spark
* Фичи можно перевести в pandas и обучить свой любимый алгоритм локально (но не обязательно)
* Предсказать им тест сет и побить бейзлайн по целевым метрикам

## Метрики.

Поскольку просмотры распределены экспоненциально, предсказывать будем странную величину log(post_views + 100). Вычисляются сразу 4 метрики, но они связаны между собой. Это
* RMSPE - Root Mean Squared Percentage Error
* MAPE - Mean Absolute Percentage Error
* MAE - Mean Absolute Error
* RMSE - Root Mean Squared Error

## Сабмиты.

За день можно делать максимум 5 сабмитов. Для вашего итогового сабмита нужно выставить флаг final=True для функции make_eval. Это сохраняет ваш сабмит для проверки, без этого задание считается не сделанным!

## Критерии.
Для получения оценки > 0 нужно прислать ноутбук в виде .ipynb файла, который можно выполнить через Run All, и который в результате сделает сабмит.
Для получения положительной оценки (>3) нужно сделать сабмит с метриками лучше, чем у бейзлайна

BASELINE = {
	'mape': 15.707128974856676,
	'mean_absolute_error': 1.219070382113261,
	'mean_squared_error': 2.4324378881170055,
	'rmse': 1.5596274837655963,
	'rmspe': 23.50065988751091
}

Выполнение этого условия дает вам оценку 5. Остальные баллы можно получить за следующие пункты:
* 1 балл за фичу, использующую метаданные каналов (макс 1 балл)
* 2 балла за фичу, использующую текстовые поля (макс 2 балла) фичи должны быть осмысленными для задачи
* 1 балл за фичу, использующую window aggregation, до 2 штук, различающихся по смыслу и реализации. Фичи которые можно альтернативно подсчитать групбаем и джойном не засчитываются (макс 2 балла)


## install stuff

In [ ]:
! pip3 install pyspark pandas scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## setup pyspark

In [ ]:
import pyspark.sql
from pyspark.sql import functions as F
import pyspark

from pyspark.sql import Window
from pyspark.ml.feature import Word2Vec, Tokenizer
from pyspark.ml.functions import vector_to_array

In [ ]:
import findspark
findspark.init()

In [ ]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "2048m")\
    .set("spark.driver.memory", "2048m")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

## load data

In [ ]:
train_data = ss.read.orc('/content/drive/MyDrive/big_data_hw2/posts_train.orc')
test_data = ss.read.orc('/content/drive/MyDrive/big_data_hw2/posts_test.orc')
channel_data = ss.read.orc('/content/drive/MyDrive/big_data_hw2/channels_orc')

# data exploration

Посмотрим, сколько всего у нас обучающих, тестовых и мета- данных:

In [ ]:
display(train_data.count())
display(test_data.count())
display(channel_data.count())

5460759

244386

2000

А как они выглядят?

In [ ]:
display(train_data.show(5))
display(test_data.show(5))
display(channel_data.show(5))

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:...|        null|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:...|        null|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:...|        null|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:...|        null|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:...|        null|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------

None

+----------+--------+-----+--------------------+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+---------+------------+--------------------+------------+
|      4127|53225416| 2137|История про песик...|        f|           f|2019-01-14 15:42:...|        null|
|      4127|53225442| 2135|Наверное, вы заме...|        f|           f|2019-01-10 11:35:...|        null|
|     34317|53042118|11612|ર્ષમાં ફેલાઈ છે જ...|        f|           f|2019-01-05 13:40:...|        null|
|     34317|53042119|39544|ર્ષમાં ફેલાઈ છે જ...|        f|           t|2019-01-05 13:40:...|    53042118|
|     34317|53042120|11611|*🎯Pls don't copy...|        f|           f|2019-01-05 13:40:...|        null|
+----------+--------+-----+--------------------+---------+------------+--------------------+------------+
only showing top 5 rows



None

+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|         description|is_private|         last_parsed|            name|post_count|     tg_id|               title|             updated|user_count|channel_id|
+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|                    |     false|2019-01-26 16:53:...|   MoeinZchannel|     708.0|1002972402|             Moein Z|2019-01-26 16:53:...|   62411.0|      7910|
|Вокруг столько ме...|     false|2019-02-12 00:39:...|  merzotachannel|    1027.0|1336284461|            Мерзость|2019-02-12 00:39:...|   12982.0|     14121|
|🗣እኛስ የተሰቀለውን ክርስ...|     false|2019-02-10 06:57:...|christian_mezmur|    1168.0|1136987361|Christian Mezmur ...|2019-02-10 06:57:...|   21704.0|     17375|
|     Chiroyli_qomatt|     false|2018-12-08 00:54:...

None

In [ ]:
channel_data.filter(F.col('channel_id')==34317).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------------------+----------+----------+----------+-----------+-----------------------+----------+----------+
|description                                                                                                                                                                                                                                        |is_private|last_parsed            |name      |post_count|tg_id     |title      |updated                |user_count|channel_id|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+

In [ ]:
train_data.printSchema()

root
 |-- channel_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- tg_id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- has_image: string (nullable = true)
 |-- is_forwarded: string (nullable = true)
 |-- date: string (nullable = true)
 |-- forwarded_id: string (nullable = true)



In [ ]:
channel_data.printSchema()

root
 |-- description: string (nullable = true)
 |-- is_private: boolean (nullable = true)
 |-- last_parsed: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- post_count: double (nullable = true)
 |-- tg_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- user_count: double (nullable = true)
 |-- channel_id: long (nullable = true)



## check features

Подберём первые фичи, которые очевидно  должны войти в признаковое пространство

In [ ]:
cols_to_analyze = ['has_image', 'is_forwarded']
for col in cols_to_analyze:
  num_col_train = train_data.select(F.col(col)).filter(F.col(col)=='t').count()
  num_col_test = test_data.select(F.col(col)).filter(F.col(col)=='t').count()
  print('train: ', col, num_col_train, '(',
                        round(num_col_train / train_data.count() * 100, 2),'%)')
  print('test: ', col, num_col_test,'(',
                        round(num_col_test / test_data.count() * 100, 2),'%)')

train:  has_image 2043958 ( 37.43 %)
test:  has_image 98825 ( 40.44 %)
train:  is_forwarded 595694 ( 10.91 %)
test:  is_forwarded 33286 ( 13.62 %)


## metadata

In [ ]:
channel_data.select('channel_id', 'updated').groupby('channel_id')\
            .agg(F.count('*').alias('num_occurences'),
                F.max('updated').alias('last_updated'),
                F.min('updated').alias('first_updated'))\
            .orderBy('num_occurences', ascending=False).show(10, 0)

+----------+--------------+-----------------------+-----------------------+
|channel_id|num_occurences|last_updated           |first_updated          |
+----------+--------------+-----------------------+-----------------------+
|45166     |1             |2019-02-13 06:23:33.062|2019-02-13 06:23:33.062|
|56242     |1             |2019-01-26 22:01:37.681|2019-01-26 22:01:37.681|
|4894      |1             |2018-12-07 07:15:24.675|2018-12-07 07:15:24.675|
|4823      |1             |2019-02-12 02:29:14.729|2019-02-12 02:29:14.729|
|11938     |1             |2018-12-11 07:35:03.549|2018-12-11 07:35:03.549|
|15194     |1             |2019-02-12 16:06:03.971|2019-02-12 16:06:03.971|
|15057     |1             |2018-12-09 10:05:50.788|2018-12-09 10:05:50.788|
|62989     |1             |2019-02-10 23:56:27.604|2019-02-10 23:56:27.604|
|42969     |1             |2018-12-08 16:17:51.693|2018-12-08 16:17:51.693|
|541       |1             |2019-02-10 01:00:38.254|2019-02-10 01:00:38.254|
+----------+

Видим, что в channel_data лежит только по одной строчке для каждого из каналов, что предполагает лик данных при использовании данной таблице (у нас как бы есть инфа о количестве подписчиков из будущего, что позволяет предсказать повышение количества просмотров к определённому моменту [нужно брать в качестве фичи количество просмотров на последнем посте, о котором эта циферка известна]), но при этом предполагает собой некотрую сложность: невозможно только по channel_data предсказать разное количество просмотров для разных постов одного и того же канала

В качестве фичей возьмём user_count & post_count как метаданные каналов

## window_func 1

А что по уникальным каналам в двух выборках?

In [ ]:
display('Уникальные каналы в train:', train_data.select('channel_id').distinct().count())
display('Уникальные каналы в test:', test_data.select('channel_id').distinct().count())

'Уникальные каналы в train:'

2000

'Уникальные каналы в test:'

1197

То есть получается, что мы можем использовать как фичу для предикта информацию о последнем количестве просмотров в трейне? Сначала надо убедиться, что данные в тесте начинаются строго позже окончания данных в трейне (мб там вообще есть лик)

In [ ]:
(train_data.groupby('channel_id')
            .agg(F.max(F.col('date')).alias('last_train'))
            .join(test_data
                        .groupby('channel_id')
                        .agg(F.min(F.col('date')).alias('first_test')),
                  on='channel_id', how='inner')
            .withColumn('days_between_train_test',
                        F.datediff(F.col('first_test'),
                                   F.col('last_train')))
            .orderBy('days_between_train_test')
).show(10, 0)

+----------+----------------------+----------------------+-----------------------+
|channel_id|last_train            |first_test            |days_between_train_test|
+----------+----------------------+----------------------+-----------------------+
|114       |2018-12-31 13:19:10+00|2019-01-01 14:34:35+00|1                      |
|184       |2018-12-31 23:33:36+00|2019-01-01 10:16:08+00|1                      |
|164       |2018-12-31 21:09:50+00|2019-01-01 07:03:55+00|1                      |
|25        |2018-12-31 22:13:23+00|2019-01-01 05:55:59+00|1                      |
|166       |2018-12-31 22:51:51+00|2019-01-01 11:26:54+00|1                      |
|47        |2018-12-31 22:07:45+00|2019-01-01 10:22:56+00|1                      |
|167       |2018-12-31 17:53:43+00|2019-01-01 13:04:16+00|1                      |
|187       |2018-12-31 17:36:56+00|2019-01-01 07:12:14+00|1                      |
|168       |2018-12-31 16:40:51+00|2019-01-01 08:39:50+00|1                      |
|42 

Видим, что действительно все тестовые посты идут только после последнего поста из трейна. Супер! Оформим в виде оконной функции выгрузку views, чтобы получить +1 за задание

In [ ]:
from pyspark.sql import Window

w1 = Window.partitionBy("channel_id").orderBy(F.desc("date"))
train_data.withColumn(
    "rn", 
    F.row_number().over(w1)
).filter("rn = 1").select(
    "channel_id", 
    "date", 
    "views"
).show(10, 0)

+----------+----------------------+-----+
|channel_id|date                  |views|
+----------+----------------------+-----+
|2         |2018-12-28 10:00:23+00|36546|
|10        |2018-12-31 15:48:27+00|631  |
|21        |2018-12-27 11:21:39+00|5764 |
|25        |2018-12-31 22:13:23+00|15291|
|28        |2018-12-06 15:19:38+00|216  |
|42        |2018-12-31 22:25:45+00|11543|
|47        |2018-12-31 22:07:45+00|21699|
|51        |2018-12-31 09:10:17+00|1634 |
|52        |2018-12-31 12:38:29+00|15845|
|70        |2018-12-31 17:53:03+00|17083|
+----------+----------------------+-----+
only showing top 10 rows



Проверим, что всё реализовано верно:

In [ ]:
train_data.filter(F.col('channel_id')==2).orderBy('date', ascending=False).show(2)

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|         2|54363783| 1426|ПРЕЗИРАЮ ТЕХ, КТО...|36546|        f|           f|2018-12-28 10:00:...|        null|
|         2|54363784| 1425|В ЭРУ ХЕНДЖОБА ДА...|29409|        f|           f|2018-12-27 14:46:...|        null|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
only showing top 2 rows



Для создания трейновых фичей нам надо смотреть на предыдущее количество просмотров, заполняя нули в самом первом посте канала количеством подписчиков из метаданных (но это уже в пандасе реализуем, чтобы не слишком сильно запариваться):

In [ ]:
w1 = Window.partitionBy("channel_id").orderBy("date")
train_data.withColumn(
    "rn", 
    F.row_number().over(w1)
).withColumn(
    'prev_views',
    F.lag('views', 1).over(w1)
).select(
    "channel_id", 
    "date", 
    'prev_views',
    "views"
).filter(F.col('channel_id')==2).show(10, 0)

+----------+----------------------+----------+-----+
|channel_id|date                  |prev_views|views|
+----------+----------------------+----------+-----+
|2         |2016-05-25 15:15:50+00|null      |415  |
|2         |2016-05-25 15:32:55+00|415       |484  |
|2         |2016-05-25 18:01:53+00|484       |398  |
|2         |2016-05-25 19:08:12+00|398       |376  |
|2         |2016-05-25 19:08:12+00|376       |394  |
|2         |2016-05-25 21:15:45+00|394       |481  |
|2         |2016-05-25 22:35:57+00|481       |451  |
|2         |2016-05-26 08:49:00+00|451       |413  |
|2         |2016-05-26 13:05:02+00|413       |409  |
|2         |2016-05-26 15:12:35+00|409       |476  |
+----------+----------------------+----------+-----+
only showing top 10 rows



## window_func 2

Долго думал, что именно надо сделать, чтобы получить полный балл за задание, удовлетворив условию "различающихся по смыслу и реализации". Не придумал ничего лучше, чем смотреть количество часов разницы между постами (ещё были вариант со скользящим средним количества просмотров последних N постов)



In [ ]:
train_data.withColumn('previous_date',
                      F.lag('date', 1).over(w1)
                      ).withColumn('hours_diff', F.datediff(F.col('date'), F.col('previous_date'))*24 + 
                          F.hour(F.col('date')) - F.hour(F.col('previous_date'))
                      ).filter(F.col('channel_id')==2).select('date', 'previous_date', 'hours_diff').orderBy('date').show(5, 0)

+----------------------+----------------------+----------+
|date                  |previous_date         |hours_diff|
+----------------------+----------------------+----------+
|2016-05-25 15:15:50+00|null                  |null      |
|2016-05-25 15:32:55+00|2016-05-25 15:15:50+00|0         |
|2016-05-25 18:01:53+00|2016-05-25 15:32:55+00|3         |
|2016-05-25 19:08:12+00|2016-05-25 18:01:53+00|1         |
|2016-05-25 19:08:12+00|2016-05-25 19:08:12+00|0         |
+----------------------+----------------------+----------+
only showing top 5 rows



Так, для первого значения теста будем брать последнее из трейна (сделаем в пандасе), а нули в трейне заполним 0

## text features

Наиболее очевидным и простым методом использования текстовых фичей является обучение фичей на основе встроенных методов обработки текста в pyspark. Но возьмём ещё и счётчик длины поста и количества слов (гипотеза - длинные\большие посты могут быть проигнорированы в популярных группах, тогда как короткие наоборот несложно читать. Да и кажется где-то было исследование, что в самых просматриваемых постах обычно 3-4 строчки, оттого у Твиттера и было ограничение на макс количество символов до 2023 года)

### IDF

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(train_data.na.drop().limit(20))
wordsData.show()

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|               words|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+
|       221|47504886| 7476|Что-то происходит...| 8896|        f|           t|2018-12-06 15:37:...|    47504885|[что-то, происход...|
|       221|47504893| 7470|Почти на месяц от...|25487|        t|           t|2018-12-06 08:56:...|    47504892|[почти, на, месяц...|
|       221|47504895| 7469|Продление домашне...| 7540|        f|           t|2018-12-06 08:31:...|    47504894|[продление, домаш...|
|       221|47504897| 7468|🎉 В нашей команд...|26107|        t|           t|2018-12-06 07:14:...|    47504896|[🎉, в, нашей, ко...|
|       221|47504900| 7466|Мне сегодня 33 го...|13902|        f|       

In [ ]:
from pyspark.ml.feature import  Tokenizer, CountVectorizer, IDF
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(train_data.na.drop())
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
# IDF down-weighs features which appear frequently in a corpus.
# This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("views", "features").show(10, 0)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
rescaledData.printSchema()

root
 |-- channel_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- tg_id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- has_image: string (nullable = true)
 |-- is_forwarded: string (nullable = true)
 |-- date: string (nullable = true)
 |-- forwarded_id: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)



### Word2Vec

Обучили IDF на трейне, но мне показалось слегка сложным переворачивать все эти спарс-векторы в нормальные эмбеддинги (всё-таки у нас не совсем ML соревнование), поэтому просто сделаем векторы из 16 значений с помощью Word2Vec

In [ ]:
from pyspark.ml.feature import Word2Vec, Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(train_data.na.drop().limit(100000))
word2Vec = Word2Vec(vectorSize=8, minCount=0, inputCol="words", outputCol="embedding")
model = word2Vec.fit(wordsData)
result = model.transform(wordsData)
result.select('views', 'words', 'embedding').show(10)

+-----+--------------------+--------------------+
|views|               words|           embedding|
+-----+--------------------+--------------------+
| 8896|[что-то, происход...|[0.64090907573699...|
|25487|[почти, на, месяц...|[0.55406442776854...|
| 7540|[продление, домаш...|[0.40907601567526...|
|26107|[🎉, в, нашей, ко...|[0.61799097994327...|
|13902|[мне, сегодня, 33...|[0.54735831943158...|
| 5360|[он, всю, жизнь, ...|[0.62124765083487...|
|16482|[потрясающий, пос...|[0.32298818246151...|
|96407|[🔥, мы, все, люб...|[0.57657012893842...|
|19182|[рассмешить, бога...|[0.47459712214767...|
| 5378|[темы, для, сочин...|[0.33720478389155...|
+-----+--------------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.ml.functions import vector_to_array

result.select('embedding').show(10, 0)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|embedding                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.6409090757369995,0.44554550014436245,0.25494077934750486,0.03182462843999798,-0.24103934677051644,0.22111785398530107,-0.31861206569841927,-0.007905179102505955]   |
|[0.5540644277685455,0.45405464233564474,0.2817830961596753,-0.051316744348566444,-0.2778834840282798,0.31932257887508186,-0.29362915296639713,-0.02785013012388455]    |
|[0.40907601567526014,0.3418647319438957,0.30454823559510613,-0.07707129723782981,-0.2518700982534116,0.3767274333344352,-0.19808456409842737,-4.15786

In [ ]:
result.select(result.columns+[vector_to_array(F.col('embedding'))[i].alias('value_'+str(i)) for i in range(8)]).show(5)

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|               words|           embedding|             value0|             value1|             value2|              value3|              value4|             value5|              value6|              value7|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|       221|47504886| 7476|Что-то происходит..

In [ ]:
model.findSynonyms('жизнь', 5).show()

+--------+------------------+
|    word|        similarity|
+--------+------------------+
|кампании|0.9986969232559204|
| недавно|0.9986495971679688|
|    этот|0.9983298778533936|
|     был|0.9983270764350891|
|  решили|0.9982300400733948|
+--------+------------------+



Видим, что эмбеддинг получился откровенно плохим, но я потратил 3 часа на то, чтобы подобрать длину эмбеддинга и размера сабсэмлпа таким, чтобы тетрадка не падала (локально оч тяжко обучать такое)

### Количество слов и символов

In [ ]:
wordsData.withColumn('num_words', F.size('words')).show(10)

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+---------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|               words|num_words|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+---------+
|       221|47504886| 7476|Что-то происходит...| 8896|        f|           t|2018-12-06 15:37:...|    47504885|[что-то, происход...|       28|
|       221|47504893| 7470|Почти на месяц от...|25487|        t|           t|2018-12-06 08:56:...|    47504892|[почти, на, месяц...|       28|
|       221|47504895| 7469|Продление домашне...| 7540|        f|           t|2018-12-06 08:31:...|    47504894|[продление, домаш...|       31|
|       221|47504897| 7468|🎉 В нашей команд...|26107|        t|           t|2018-12-06 07:14:...|    47504896|[🎉, в, нашей, ко...|       63|
|

In [ ]:
wordsData.withColumn('num_symbols', F.length('text')).show(10)

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+-----------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|               words|num_symbols|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+--------------------+-----------+
|       221|47504886| 7476|Что-то происходит...| 8896|        f|           t|2018-12-06 15:37:...|    47504885|[что-то, происход...|        163|
|       221|47504893| 7470|Почти на месяц от...|25487|        t|           t|2018-12-06 08:56:...|    47504892|[почти, на, месяц...|        171|
|       221|47504895| 7469|Продление домашне...| 7540|        f|           t|2018-12-06 08:31:...|    47504894|[продление, домаш...|        217|
|       221|47504897| 7468|🎉 В нашей команд...|26107|        t|           t|2018-12-06 07:14:...|    47504896|[🎉, в, нашей, ко...|

# make features

In [ ]:
train_data.select(train_data.columns).show(5)

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:...|        null|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:...|        null|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:...|        null|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:...|        null|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:...|        null|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------

In [ ]:
def make_features(df, model, is_train = True):
    cols = ['id', 'has_image', 'is_forwarded', 'post_count', 'user_count', 'previous_views', 'hours_diff', 'num_words', 'num_symbols']+['value_'+str(i) for i in range(8)]
    if 'views' in df.columns:
      cols.append('views')
    # get window_functions
    if is_train:
      w1 = Window.partitionBy("channel_id").orderBy("date")
      df = df.withColumn(
                          'previous_views',
                          F.lag('views', 1).over(w1)
                          ).withColumn(
                              'previous_date',
                              F.lag('date', 1).over(w1)
                          ).withColumn(
                              'hours_diff', F.datediff(F.col('date'), F.col('previous_date'))*24 + 
                              F.hour(F.col('date')) - F.hour(F.col('previous_date'))
                          ).fillna(0, subset='hours_diff')
      
    else:
      w1 = Window.partitionBy("channel_id").orderBy(F.desc("date"))
      df_prev_views = train_data.withColumn(
          "rn", 
          F.row_number().over(w1)
      ).filter("rn = 1").select(
          "channel_id", 
          F.col("views").alias('previous_views')
      )
      df = df.hint('merge').join(df_prev_views, on='channel_id', how='left')

      w2 = Window.partitionBy("channel_id").orderBy("date")
      df = df.withColumn(
                          'previous_date',
                          F.lag('date', 1).over(w2)
                        ).withColumn(
                          'hours_diff', F.datediff(F.col('date'), F.col('previous_date'))*24 + 
                          F.hour(F.col('date')) - F.hour(F.col('previous_date'))
                        )
    # get text data
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    wordsData = tokenizer.transform(df.fillna('', subset='text'))
    df = model.transform(wordsData)
    df = df.withColumn('num_words', F.size('words'))\
           .withColumn('num_symbols', F.length('text'))
    df = df.select(df.columns+[vector_to_array(F.col('embedding'))[i].alias('value_'+str(i)) for i in range(8)])

    # get metadata
    return df.hint('MERGE').join(channel_data.select('channel_id', 'post_count', 'user_count'),
                                 on='channel_id', how='left').select(*cols)

In [ ]:
train_features = make_features(train_data, model).cache()
test_features = make_features(test_data, model, is_train=False).cache()

In [ ]:
train_features.write.csv('train_csv', mode='overwrite', header=True)
test_features.write.csv('test_csv', mode='overwrite', header=True)

# load features to pandas
you also can use .toPandas()

In [ ]:
import subprocess
import glob
import os
import shutil
import pandas as pd

def load_and_merge_csv(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [ ]:
trainXY = load_and_merge_csv('train_csv')
testX = load_and_merge_csv('test_csv')

In [ ]:
assert len(trainXY) == 5460759
assert len(testX) == 244386

In [ ]:
trainXY.to_csv('trainXY.csv')
testX.to_csv('testX.csv')

In [ ]:
import numpy as np

Ycol = 'views'
#to_drop = ['channel_id']
trainX, trainY = trainXY.drop(Ycol, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)
trainX['previous_views'] = np.log(trainX['previous_views']+100)
testX['previous_views'] = np.log(testX['previous_views']+100)

testX

,has_image,is_forwarded,post_count,user_count,previous_views,hours_diff,num_words,num_symbols,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7
id,,,,,,,,,,,,,,,,
55551719,f,f,3263.0,22031.0,7.828835,NaN,82,263,0.063041,-0.053459,0.097793,0.262953,0.451420,-0.302397,0.202642,0.069636
55551718,f,f,3263.0,22031.0,7.828835,2.0,364,2072,0.245435,0.116299,-0.057973,0.413628,0.459964,-0.715440,0.089699,0.401273
55551717,t,f,3263.0,22031.0,7.828835,1.0,14,82,0.156229,-0.218294,0.358031,0.572588,0.606249,-0.051025,0.065492,0.138971
55551716,f,f,3263.0,22031.0,7.828835,21.0,82,265,0.056489,-0.051801,0.080355,0.248718,0.436097,-0.306593,0.209770,0.069361
55551715,f,f,3263.0,22031.0,7.828835,3.0,193,1285,0.365334,0.124548,-0.020882,0.507938,0.483123,-0.904052,0.049493,0.538547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54339831,t,f,5520.0,31307.0,9.307649,1.0,2,21,0.123210,-0.299690,0.315246,0.155282,0.421593,-0.147286,-0.045698,-0.020093
54339830,f,f,5520.0,31307.0,9.307649,2.0,54,323,-0.082001,-0.280044,0.285495,-0.154134,-0.528002,-0.461804,0.652957,0.004114
54339829,t,f,5520.0,31307.0,9.307649,0.0,2,21,0.123210,-0.299690,0.315246,0.155282,0.421593,-0.147286,-0.045698,-0.020093


In [ ]:
trainX_dummy = pd.get_dummies(trainX)
testX_dummy = pd.get_dummies(testX)

## Заполняем пропуски, образованные оконными функциями

Они остались в трейне у prev_views и в тесте у hour_diff

In [ ]:
testX_dummy.loc[testX_dummy.isna().any(axis=1), 'hours_diff'] = 1

In [ ]:
trainX_dummy.loc[trainX_dummy.isna().any(axis=1), 'previous_views'] = trainX.user_count

In [ ]:
display(trainX_dummy.head())
display(testX_dummy.head())

,post_count,user_count,previous_views,hours_diff,num_words,num_symbols,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,has_image_f,has_image_t,is_forwarded_f,is_forwarded_t
id,,,,,,,,,,,,,,,,,,
36767705,1740.0,5239.0,5239.000000,0,564,3367,0.584264,0.442228,0.285028,-0.041272,-0.281022,0.292980,-0.283847,-0.026415,1,0,1,0
36767704,1740.0,5239.0,7.042286,0,1,0,0.127917,0.008344,-0.088678,0.407034,0.486937,-0.530228,0.131235,0.153080,0,1,1,0
36767703,1740.0,5239.0,6.511745,4,14,92,0.468187,0.334175,0.215203,-0.060529,-0.217840,0.239585,-0.205511,0.005329,1,0,1,0
36767701,1740.0,5239.0,6.442540,0,1,0,0.127917,0.008344,-0.088678,0.407034,0.486937,-0.530228,0.131235,0.153080,0,1,1,0
36767702,1740.0,5239.0,6.426488,0,1,0,0.127917,0.008344,-0.088678,0.407034,0.486937,-0.530228,0.131235,0.153080,0,1,1,0


,post_count,user_count,previous_views,hours_diff,num_words,num_symbols,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,has_image_f,has_image_t,is_forwarded_f,is_forwarded_t
id,,,,,,,,,,,,,,,,,,
55551719,3263.0,22031.0,7.828835,1.0,82,263,0.063041,-0.053459,0.097793,0.262953,0.451420,-0.302397,0.202642,0.069636,1,0,1,0
55551718,3263.0,22031.0,7.828835,2.0,364,2072,0.245435,0.116299,-0.057973,0.413628,0.459964,-0.715440,0.089699,0.401273,1,0,1,0
55551717,3263.0,22031.0,7.828835,1.0,14,82,0.156229,-0.218294,0.358031,0.572588,0.606249,-0.051025,0.065492,0.138971,0,1,1,0
55551716,3263.0,22031.0,7.828835,21.0,82,265,0.056489,-0.051801,0.080355,0.248718,0.436097,-0.306593,0.209770,0.069361,1,0,1,0
55551715,3263.0,22031.0,7.828835,3.0,193,1285,0.365334,0.124548,-0.020882,0.507938,0.483123,-0.904052,0.049493,0.538547,1,0,1,0


# train your model and predict test

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
BASELINE = {
	'mape': 15.707128974856676,
	'mean_absolute_error': 1.219070382113261,
	'mean_squared_error': 2.4324378881170055,
	'rmse': 1.5596274837655963,
	'rmspe': 23.50065988751091
}


In [ ]:
def calc_metrics(y_test, y_test_pred, BASELINE):
  result = {'mape': mean_absolute_percentage_error(y_test, y_test_pred),
            'mean_absolute_error': mean_absolute_error(np.array(y_test), y_test_pred),
            'mean_squared_error': mean_squared_error(y_test, y_test_pred),
            'rmse': mean_squared_error(y_test, y_test_pred, squared=False),
            'rmspe': np.sqrt(np.mean(np.square(((y_test - y_test_pred) / y_test)), axis=0))}
  print("Test RMSE = %.4f" % result['mape'])
  print("Test MSE = %.4f" % result['mean_absolute_error'])
  print("Test RMSPE = %.4f" % result['mean_squared_error'])
  print("Test MAPE = %.4f" % result['rmse'])
  print("Test MAE = %.4f" % result['rmspe'])
  print({key: result[key]/BASELINE.get(key, 0) for key in BASELINE})

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(trainX_dummy, trainY, test_size=0.2, random_state=12)

In [ ]:
prediction = np.random.random(testX.shape[0])

In [ ]:
prediction.shape

In [ ]:
assert prediction.shape == (244386,)

## linear models

### Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
# make predictions
y_val_pred = lr.predict(X_val)
calc_metrics(y_val, y_val_pred, BASELINE)

Test RMSE = 0.1269
Test MSE = 0.9384
Test RMSPE = 1.6168
Test MAPE = 1.2715
Test MAE = 0.1931
{'mape': 0.008081193424885757, 'mean_absolute_error': 0.7697785136304144, 'mean_squared_error': 0.6646703619974299, 'rmse': 0.8152731824348387, 'rmspe': 0.008216992767183955}


In [ ]:
prediction = lr.predict(testX_dummy)

### Lasso

In [ ]:
lr = Lasso()
lr.fit(X_train, y_train)
# make predictions
y_val_pred = lr.predict(X_val)
calc_metrics(y_val, y_val_pred, BASELINE)

Test RMSE = 0.1322
Test MSE = 0.9825
Test RMSPE = 1.7259
Test MAPE = 1.3137
Test MAE = 0.1947
{'mape': 0.00841519332298613, 'mean_absolute_error': 0.8059455966899984, 'mean_squared_error': 0.7095455306529628, 'rmse': 0.8423452562061252, 'rmspe': 0.008284534047732602}


### Ridge

In [ ]:
lr = Ridge()
lr.fit(X_train, y_train)
# make predictions
y_val_pred = lr.predict(X_val)
calc_metrics(y_val, y_val_pred, BASELINE)

Test RMSE = 0.1269
Test MSE = 0.9384
Test RMSPE = 1.6168
Test MAPE = 1.2715
Test MAE = 0.1931
{'mape': 0.008081193440910465, 'mean_absolute_error': 0.7697785125795841, 'mean_squared_error': 0.6646703581810445, 'rmse': 0.8152731800942825, 'rmspe': 0.008216992661572275}


## tree based

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

### catboost

In [ ]:
from sklearn.model_selection import ParameterSampler
from tqdm import tqdm

params = {'learning_rate': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [1, 3],
        'iterations': [100, 200, 300]}
param_list = list(ParameterSampler(param_distributions=params, n_iter=10, random_state=21))
mae = np.zeros_like(np.arange(len(param_list)), dtype=float)
print(param_list[0].keys())

with tqdm(total=len(param_list)-1) as pbar:
    for i in range(len(param_list)):
        print(param_list[i].values())
        values = [*param_list[i].values()]
        cb_reg = CatBoostRegressor(depth=values[3], l2_leaf_reg=values[1], 
                                    iterations=values[2], learning_rate=values[0],
                                    random_state=21)
        cb_reg.fit(X=X_train, y=y_train, plot=False,
                   eval_set=[(X_val, y_val)], 
                  verbose=False)
        y_pred = cb_reg.predict(X_val)
        calc_metrics(y_val, y_pred, BASELINE)

dict_keys(['learning_rate', 'l2_leaf_reg', 'iterations', 'depth'])


  0%|          | 0/9 [00:00<?, ?it/s]

dict_values([0.1, 1, 100, 4])


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Test RMSE = 0.0384
Test MSE = 0.2897
Test RMSPE = 0.4850
Test MAPE = 0.6964
Test MAE = 0.1049
{'mape': 0.002447274511496172, 'mean_absolute_error': 0.23762835647701222, 'mean_squared_error': 0.19937187226134948, 'rmse': 0.44651077507866427, 'rmspe': 0.004463776747493109}
dict_values([0.2, 1, 300, 8])


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

  0%|          | 0/9 [06:04<?, ?it/s]


KeyboardInterrupt: ignored

### sklearn gradient boosting

In [ ]:
lr = GradientBoostingRegressor()
lr.fit(X_train, y_train)
# make predictions
y_val_pred = lr.predict(X_val)
calc_metrics(y_val, y_val_pred, BASELINE)

### sklearn rf


In [ ]:
lr = RandomForestRegressor()
lr.fit(X_train, y_train)
# make predictions
y_val_pred = lr.predict(X_val)
calc_metrics(y_val, y_val_pred, BASELINE)

## final model

In [ ]:
cb_reg = CatBoostRegressor(depth=4, l2_leaf_reg=1, 
                                    iterations=100, learning_rate=0.1,
                                    random_state=21)
cb_reg.fit(X=X_train, y=y_train, plot=False,
          eval_set=[(X_val, y_val)], 
        verbose=False)
y_pred = cb_reg.predict(X_val)
calc_metrics(y_val, y_pred, BASELINE)

Test RMSE = 0.0384
Test MSE = 0.2897
Test RMSPE = 0.4850
Test MAPE = 0.6964
Test MAE = 0.1049
{'mape': 0.002447274511496172, 'mean_absolute_error': 0.23762835647701222, 'mean_squared_error': 0.19937187226134948, 'rmse': 0.44651077507866427, 'rmspe': 0.004463776747493109}


In [ ]:
prediction = cb_reg.predict(testX_dummy)

## additional: Vowpal Wabbit usage

Хотел показать, что можно обучать модели с помощью инструмента для больших данных, но стало немного лень. Просто оставлю возможные UseCase для либы Vowpal Wabbit [здесь](https://github.com/ZolotarevStat/University/blob/main/ML%20on%20Big%20Data/BigHW_Outbrain_Click_Prediction.ipynb)

# submit

In [ ]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2023/src/client.py -o client.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1907  100  1907    0     0   8747      0 --:--:-- --:--:-- --:--:--  8747


In [ ]:
import client

In [ ]:
client.make_eval(pd.DataFrame({'views': prediction}, index=testX.index), final=True)

{'data': {'mape': 8.918192891620633,
  'mean_absolute_error': 0.6815061335366615,
  'mean_squared_error': 1.3558900257916644,
  'rmse': 1.1644269087373686,
  'rmspe': 17.39001495991588},
 'ok': True}

In [ ]:
client.check_results()

{'2022-06-22': [{'baseline_beaten': True,
   'is_final': False,
   'metrics': {'mape': 12.727620679181136,
    'mean_absolute_error': 0.9455263275003919,
    'mean_squared_error': 1.7195696729463468,
    'rmse': 1.3113236339463827,
    'rmspe': 20.916288967369184}},
  {'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 13.451678827060269,
    'mean_absolute_error': 0.9721868654662573,
    'mean_squared_error': 1.929836459199178,
    'rmse': 1.389185538075882,
    'rmspe': 23.510528950926457}},
  {'baseline_beaten': True,
   'is_final': False,
   'metrics': {'mape': 11.05639899081406,
    'mean_absolute_error': 0.8431678590759435,
    'mean_squared_error': 1.356104486717572,
    'rmse': 1.164518993712671,
    'rmspe': 17.512251896373918}},
  {'baseline_beaten': True,
   'is_final': True,
   'metrics': {'mape': 11.05639899081406,
    'mean_absolute_error': 0.8431678590759435,
    'mean_squared_error': 1.356104486717572,
    'rmse': 1.164518993712671,
    'rmspe': 17.5